In [ ]:
# !pip install openai

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

WEAVIATE_HTTP_URL = os.getenv("WEAVIATE_URL")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_URL = os.getenv("OPENAI_URL")

USERNAME = os.getenv("USERNAME")
WIKI_NAME = f"{USERNAME}_wiki"

print(WEAVIATE_HTTP_URL, OPENAI_URL, OPENAI_API_KEY)
print(WIKI_NAME)

## Generate query from prompt

In [ ]:
from openai import OpenAI
openai_client = OpenAI(
    api_key=OPENAI_API_KEY,
    base_url=f"{OPENAI_URL}/v1"
)

In [ ]:
def generate_query_from_promt(prompt):
    response = openai_client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            { 
                "role": "system",
                "content": "Your job is to extract a query from the provided user prompt, the query will then be used to run a vector search in a vector database." },
            { 
                "role": "user",
                "content": f"Please give me a 2-3 word query that can be used to find relevant info to the following prompt - {prompt}"
            },
        ]
    )
    return response.choices[0].message.content

In [ ]:
# Example of how to generate a query from a prompt
generate_query_from_promt("Where do the tallest penguins live?")

## Connect to Weaviate

In [ ]:
import weaviate
from weaviate.classes.init import AdditionalConfig, Timeout

client = weaviate.connect_to_local(
    host=WEAVIATE_HTTP_URL,
    headers={
        "X-OpenAI-Api-Key": OPENAI_API_KEY,
    },

    # connect with timeout config
    additional_config=AdditionalConfig(
        timeout=Timeout(query=60)  # Values in seconds
    )
)

client.is_ready()

## Two-step RAG

In [ ]:
# from weaviate.classes.generate import GenerativeConfig

BLUE   = "\033[94m"
PURPLE = "\033[95m"
RESET  = "\033[0"

def two_step_rag(user_prompt):
    # Step 1
    prompt = user_prompt + " Please only use the provided content with this prompt. Don't make things up."
    
    generated_query = generate_query_from_promt(prompt)
    print("=== Generated Query ===")
    print(f"Generated query: {generated_query}")

    # Step 2
    wiki = client.collections.get(WIKI_NAME)

    response = wiki.generate.near_text(
        query=generated_query,
        limit=3,

        # generative_provider=GenerativeConfig.openai(
        #     # model="gpt-4o-mini-0718",
        #     model="gpt-4o-mini",
        # ),

        grouped_task=prompt,
        grouped_properties=["text", "title"],
    )

    # Print results
    print(f"\n{BLUE}=== Generated Response ===")
    print(response.generative.text)

    print(f"\n{PURPLE}=== Source ===")
    for item in response.objects:
        print(item.properties)

In [ ]:
two_step_rag("How do airplanes fly? Help me write a 3 paragraph assignment for my school project. Oh btw, I am 10 years old.")

In [ ]:
two_step_rag("What are the pros and cons of automation using computer?")

In [ ]:
two_step_rag("How do CPUs work?")

## Close the client

In [ ]:
client.close()